<a href="https://colab.research.google.com/github/Nekhaenko/test/blob/main/%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colab!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import autogen  # Импорт библиотеки AutoGen для создания многоагентных систем

In [2]:
# Конфигурация для локального сервера Ollama, определяющая параметры модели
config_list = [
    {
        "model": "codellama:7b",  # Используем модель CodeLLaMA с 7 миллиардами параметров для генерации кода
        "api_type": "ollama",  # Интерфейс API для подключения к Ollama
        "client_host": "http://localhost:11434",  # Локальный адрес сервера Ollama
        "num_predict": 1024,  # Максимальное количество токенов в ответе
        "temperature": 0.1,  # Низкая температура для детерминированных ответов
        "top_p": 0.95,  # Топ-95% вероятных токенов для генерации
        "stream": False  # Отключение потоковой передачи ответа
    }
]

# Определение агента: Менеджер продукта
product_manager = autogen.ConversableAgent(
    name="product_manager",  # Уникальное имя агента
    system_message="""Вы продукт менеджер, который принимает идею и описывает план действий,
которые должен реализовать software_engineer""",  # Инструкция: формировать план для разработчика
    llm_config={"config_list": config_list},  # Конфигурация модели
    human_input_mode="NEVER",  # Полностью автономная работа без вмешательства человека
)

In [3]:
# Определение агента: Разработчик
software_engineer = autogen.ConversableAgent(
    name="software_engineer",  # Уникальное имя агента
    system_message="Вы разработчик, который принимает план действий от product_manager и пишите код",  # Инструкция: писать код по плану
    llm_config={"config_list": config_list},  # Конфигурация модели
    human_input_mode="NEVER",  # Автономная работа
)

# Определение агента: Специалист по тестированию
qa_engineer = autogen.ConversableAgent(
    name="qa_engineer",  # Уникальное имя агента
    system_message="Вы специалист по тестированию, вы принимаете код от software_engineer и пишите unit tests",  # Инструкция: создавать юнит-тесты
    llm_config={"config_list": config_list},  # Конфигурация модели
    human_input_mode="NEVER",  # Автономная работа
)

# Определение агента: DevOps-инженер
devops_engineer = autogen.ConversableAgent(
    name="devops_engineer",  # Уникальное имя агента
    system_message="""Вы devops инженер, который учитывает контекст от product_manager, software_engineer, qa_engineer
и реализовываете CI/CD""",  # Инструкция: разрабатывать CI/CD на основе работы других агентов
    llm_config={"config_list": config_list},  # Конфигурация модели
    human_input_mode="NEVER",  # Автономная работа
)

# Запуск последовательных чатов, инициируемых product_manager
chat_results = product_manager.initiate_chats(
    [
        {
            "recipient": software_engineer,  # Получатель: разработчик
            "message": """Реализовать программу, которая получает данные из временой таблицы,
удаляет дубликаты и записывает результат в историческую таблицу""",  # Задача: написать программу для обработки таблиц
            "max_turns": 1,  # Один обмен сообщениями (запрос-ответ)
            "summary_method": "last_msg",  # Суммаризация: использовать последнее сообщение
        },
        {
            "recipient": qa_engineer,  # Получатель: тестировщик
            "message": """Вот мои unit tests""",  # Сообщение: запрос на написание тестов (контекст берётся из предыдущего чата)
            "max_turns": 1,  # Один обмен
            "summary_method": "last_msg",  # Суммаризация: последнее сообщение
        },
        {
            "recipient": devops_engineer,  # Получатель: DevOps-инженер
            "message": """Вот моя реализация CI/CD""",  # Сообщение: запрос на настройку CI/CD (контекст из предыдущих чатов)
            "max_turns": 1,  # Один обмен
            "summary_method": "last_msg",  # Суммаризация: последнее сообщение
        },
    ]
)

# Вывод результатов чатов
print("\n\nFirst Chat Summary (software_engineer):\n", chat_results[0].summary)  # Вывод суммаризации первого чата (код от разработчика)
print("\n\nSecond Chat Summary (qa_engineer):\n", chat_results[1].summary)  # Вывод суммаризации второго чата (юнит-тесты)
print("\n\nThird Chat Summary (devops_engineer):\n", chat_results[2].summary)  # Вывод суммаризации третьего чата (CI/CD)


********************************************************************************
Starting a new chat....

********************************************************************************
product_manager (to software_engineer):

Реализовать программу, которая получает данные из временой таблицы,
удаляет дубликаты и записывает результат в историческую таблицу

--------------------------------------------------------------------------------
software_engineer (to product_manager):


Для реализации этой задачи мы можем использовать следующий код:
```
import pandas as pd

# Создаем временную таблицу с данными
data = {'name': ['John', 'Mary', 'John', 'Jane'],
        'age': [25, 30, 35, 40]}
df = pd.DataFrame(data)

# Удаляем дубликаты из временной таблицы
df_unique = df.drop_duplicates()

# Записываем результат в историческую таблицу
df_unique.to_sql('history', con=engine, if_exists='append')
```
В этом коде мы используем библиотеку pandas для работы с данными и создаем временную таблицу `d